In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing
!pip install unidecode
import unidecode
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#with open("drive/MyDrive/rnn_dataset/victorhugo.txt","r") as f:
  #text = f.read()

path_to_file = "drive/MyDrive/rnn_dataset/victorhugo.txt"

text = open(path_to_file, mode = 'rb').read().decode(encoding = 'utf-8')
print(len(text))
print(text[:1000])

127286
Parce que, jargonnant vêpres, jeûne et vigile,
Exploitant Dieu qui rêve au fond du firmament,
Vous avez, au milieu du divin évangile,
Ouvert boutique effrontément ;

Parce que vous feriez prendre à Jésus la verge,
Cyniques brocanteurs sortis on ne sait d'où ;
Parce que vous allez vendant la sainte vierge
Dix sous avec miracle, et sans miracle un sou ;

Parce que vous contez d'effroyables sornettes
Qui font des temples saints trembler les vieux piliers ;
Parce que votre style éblouit les lunettes
Des duègnes et des marguilliers ;

Parce que la soutane est sous vos redingotes,
Parce que vous sentez la crasse et non l'œillet,
Parce que vous bâclez un journal de bigotes
Pensé par Escobar, écrit par Patouillet ;

Parce qu'en balayant leurs portes, les concierges
Poussent dans le ruisseau ce pamphlet méprisé ;
Parce que vous mêlez à la cire des cierges
Votre affreux suif vert-de-grisé ;

Parce qu'à vous tout seuls vous faites une espèce
Parce qu'enfin, blanchis dehors et noirs dedans,

Preprocessing : 

enlever les majuscules 
enlever les points virgules 
ect

In [ ]:

"""
text = unidecode.unidecode(text)
"""
text = text.lower()

text = text.replace("2"," ")
text = text.replace("1"," ")
text = text.replace("8"," ")
text = text.replace("5"," ")
text = text.replace(">"," ")
text = text.replace("<"," ")
text = text.replace("!"," ")
text = text.replace("?"," ")
text = text.replace("-"," ")
text = text.replace("$"," ")
text = text.replace(";"," ")


text=text.strip()


vocab = sorted(set(text))
print( '{} caracteres uniques '.format(len(vocab)))


51 caracteres uniques 


transformer toutes les lettre en nombres 

In [ ]:
"""
vocab_size = len(vocab)

vocab_to_int = {l:i for i,l in enumerate(vocab)}
int_to_vocab = {l:i for i,l in enumerate(vocab)}

print("vocab_to_int :", vocab_to_int)
print()
print("int_to_vocab :", int_to_vocab )


print("\nint for e : ", vocab_to_int["e"])
int_for_e = vocab_to_int["e"]
#print("letter for %s : %s" % (vocab_to_int["e"],int_to_vocab[int_for_e]))
print(int_to_vocab[int_for_e])
"""
chars = tf.strings.unicode_split(text, input_encoding='UTF-8') #vectorisation

vocab_to_int = preprocessing.StringLookup(vocabulary = list(vocab))
ids = vocab_to_int(chars)
print("ids :",ids)
int_to_vocab = preprocessing.StringLookup(vocabulary = vocab_to_int.get_vocabulary(), invert=True)
chars = int_to_vocab(ids)
print("chars : ", chars)

def text_from_chars(chars):
  return tf.strings.reduce_join(chars, axis=-1)



ids : tf.Tensor([24  9 26 ... 29 26  7], shape=(127285,), dtype=int64)
chars :  tf.Tensor([b'p' b'a' b'r' ... b'u' b'r' b'.'], shape=(127285,), dtype=string)


In [ ]:
print(vocab_to_int("e"))
print(int_to_vocab(13))

tf.Tensor(13, shape=(), dtype=int64)
tf.Tensor(b'e', shape=(), dtype=string)


In [ ]:
ids_dataset = tf.dataset = tf.data.Dataset.from_tensor_slices(ids)

In [ ]:
for ids in ids_dataset.take(10):
  print(int_to_vocab(ids).numpy().decode('UTF-8'))

p
a
r
c
e
 
q
u
e
,


cette fonction va permettre au reseau d anticiper ce que va etre la prochaine
lettre a predire afin de realiser l entrainnement 

In [ ]:
def split_input_target(sequence):
  inputs = sequence[:-1]
  targets = sequence[1:]
  return inputs, targets

In [ ]:
split_input_target(list("tensorflow"))

(['t', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

plus il y a de sequence dans notre batch plus l entrainnement va etre efficace

In [ ]:
def generation_batch (inputs, targets, seq_lenth, batch_size, noise=0 ):
  #taille de chaque chunk
  chunk_size = (len(inputs) -1)//batch_size
  sequence_per_chunk = chunk_size //seq_lenth

  for s in range(0, sequence_per_chunk):
    batch_inputs = np.zeros((batch_size, seq_lenth))
    batch_targets = np.zeros((batch_size, seq_lenth))
    for b in range(0, batch_size):
      fr = (b*chunk_size)+(s*seq_lenth)
      to = fr+seq_lenth

      batch_inputs[b] = inputs[fr:to]
      batch_targets[b] = inputs[fr+1:to+1]

      if noise > 0:
        noise_indices = np.random.choice(seq_lenth, noise)
        batch_inputs[b][noise_indices] = np.random.randint(0, vocab_size)

    yield batch_inputs, batch_targets
    
for batch_inputs, batch_targets in generation_batch(inputs, target, 5, 32, noise=0):
  print(batch_inputs[0], batch_targets[0])
  break







